In [1]:
# Import libraries

# Data processing and manipulation
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

from typing import Iterable, Any, Tuple, Dict


# Machine learning models
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, log_loss, average_precision_score, balanced_accuracy_score, accuracy_score
from pathlib import Path


# Custom models
import importlib
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../src')))

import preprocess_data as ppd
import GAMinferenceModels_V2 as gam_models

# Data

In [2]:
weather_data_file = "../Data/hourly/hourly_weather_by_state.csv"
power_load_file = "../Data/hourly/hourly_load_by_state.csv"
failure_data_file = "../Data/hourly/hourly_failure_dataset_compressed.csv"

all_data_df, _, feature_names, target_columns, integer_encoding = ppd.preprocess_data(failure_data_path=failure_data_file,
                                                                                        weather_data_path=weather_data_file,
                                                                                        power_load_data_path=power_load_file,
                                                                                        feature_names=['Temperature', 'Relative_humidity', 'Load', 'State'],
                                                                                        cyclic_features=["Season", "Month", "DayOfWeek", "DayOfYear"],
                                                                                        state_one_hot=False,
                                                                                        initial_MC_state_filter='all',
                                                                                        technology_filter=['Gas Turbine/Jet Engine (Simple Cycle Operation)'],
                                                                                        test_periods=None
                                                                                        )

In [3]:
# temporal features for regional classifiers
all_data_df['month_sin'] = np.sin(2*np.pi*all_data_df['Datetime_UTC'].dt.month/12)
all_data_df['month_cos'] = np.cos(2*np.pi*all_data_df['Datetime_UTC'].dt.month/12)

# Get list of states from one-hot encoded columns
idx2state = {v: k for k, v in integer_encoding['States'].items()}
all_data_df['State'] = all_data_df['State'].apply(lambda x: idx2state[x])
states_list = all_data_df['State'].unique().tolist()
states_list.sort()

In [4]:
print("States considered:", states_list)

States considered: ['ALABAMA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING']


### 2022-2023 test

In [5]:
# general_test_period = [(pd.Timestamp('2022-01-01 00:00:00'), pd.Timestamp('2023-12-31 23:00:00'))]
general_test_period = [(pd.Timestamp('2022-01-01 00:00:00', tz='UTC'), pd.Timestamp('2023-12-31 23:00:00', tz='UTC'))]

specific_test_periods_per_state = {state: general_test_period for state in states_list}

folder_name = "2022_2023_test_periods"

### Test on extreme event

In [6]:
def detect_extreme_temperature_events(
    all_data_df: pd.DataFrame,
    states_list,
    *,
    temp_col="Temperature",
    datetime_col="Datetime_UTC",
    state_col="State",
    cold_percentile=5,
    hot_percentile=85,
    min_duration=2,
):
    """
    Detect extreme cold and hot events per state.

    Definitions
    -----------
    - Daily mean temperature computed from hourly data
    - Extreme cold: daily mean <= cold_percentile
    - Extreme hot : daily mean >= hot_percentile
    - Event = at least `min_duration` consecutive days
    - Events returned as (start_day, end_day), inclusive

    Returns
    -------
    extreme_cold_events_dt : dict[state -> list[(start_day, end_day)]]
    extreme_hot_events_dt  : dict[state -> list[(start_day, end_day)]]
    """

    extreme_cold_events_dt = defaultdict(list)
    extreme_hot_events_dt  = defaultdict(list)

    for state in states_list:
        # --- isolate state data ---
        state_data = (
            all_data_df.loc[all_data_df[state_col] == state, [datetime_col, temp_col]]
            .drop_duplicates(subset=[datetime_col])
            .sort_values(datetime_col)
            .copy()
        )

        if state_data.empty:
            continue

        # --- compute daily mean temperature ---
        state_data["Day"] = state_data[datetime_col].dt.floor("D")
        daily = (
            state_data
            .groupby("Day", as_index=False)[temp_col]
            .mean()
            .rename(columns={temp_col: "Daily_mean_Temperature"})
            .sort_values("Day")
        )

        temps = daily["Daily_mean_Temperature"].to_numpy()

        # --- thresholds ---
        cold_thr = np.percentile(temps, cold_percentile)
        hot_thr  = np.percentile(temps, hot_percentile)

        is_cold = temps <= cold_thr
        is_hot  = temps >= hot_thr

        # --- helper to extract runs ---
        def extract_runs(mask, days, min_len):
            """
            mask : boolean array
            days : array of day timestamps
            """
            runs = []
            d = np.diff(np.r_[0, mask.astype(int), 0])
            starts = np.where(d == 1)[0]
            ends   = np.where(d == -1)[0]  # exclusive

            for s, e in zip(starts, ends):
                if (e - s) >= min_len:
                    runs.append((days[s], days[e - 1]))  # inclusive end
            return runs

        days = daily["Day"].to_numpy()

        # --- extract events ---
        extreme_cold_events_dt[state] = extract_runs(
            is_cold, days, min_duration
        )
        extreme_hot_events_dt[state] = extract_runs(
            is_hot, days, min_duration
        )

    return extreme_cold_events_dt, extreme_hot_events_dt

def select_extreme_events(
        all_extreme_events: list[Tuple[pd.Timestamp, pd.Timestamp]],
        method: str = 'longest',
        n: int = 2
    ) -> list[Tuple[pd.Timestamp, pd.Timestamp]]:
    if method == 'longest':
        # Select the n longest events
        selected_events = sorted(all_extreme_events, key=lambda x: x[1] - x[0], reverse=True)[:n]
    else:
        raise ValueError(f"Invalid method {method}. Choose 'longest'.")
    return selected_events

In [ ]:
# n_cold = 1
# n_hot = 1

# extreme_cold_events, extreme_hot_events = detect_extreme_temperature_events(
#     all_data_df,
#     states_list,
#     temp_col="Temperature",
#     datetime_col="Datetime_UTC",
#     state_col="State",
#     cold_percentile=5,
#     hot_percentile=95,
#     min_duration=2,
# )

# selected_cold_events = {
#     state: select_extreme_events(events, method='longest', n=n_cold)
#     for state, events in extreme_cold_events.items()}
# selected_hot_events = {
#     state: select_extreme_events(events, method='longest', n=n_hot)
#     for state, events in extreme_hot_events.items()}

# round_of_cold_events = {i: {state: [events[i]] for state, events in selected_cold_events.items() if events} for i in range(n_cold)}
# round_of_hot_events = {i: {state: [events[i]] for state, events in selected_hot_events.items() if events} for i in range(n_hot)}

# folder_name_cold = "Extreme_cold_events"
# folder_name_hot = "Extreme_hot_events"

# Naive model

### Train-test split

In [11]:
train_sets = {}
test_sets = {}

for state in states_list:
    test_start, test_end = specific_test_periods_per_state[state][0][0], specific_test_periods_per_state[state][0][1]
    train_df = all_data_df.loc[
        (all_data_df['State'] == state) &
        (~all_data_df['Datetime_UTC'].between(test_start, test_end)),
    ].copy()

    test_df = all_data_df.loc[
        (all_data_df['State'] == state) &
        (all_data_df['Datetime_UTC'].between(test_start, test_end)),
    ].copy()

    train_sets[state] = train_df
    test_sets[state] = test_df

### Compute empirical transition frequency, and export tests

In [15]:
for state in tqdm(states_list):
    train_df = train_sets[state]
    test_df = test_sets[state].copy()

    probs_naive = {}
    for initial_state in [0,1,2]:
        df_init = train_df.loc[train_df['Initial_gen_state'] == initial_state]
        for final_state in [0,1,2]:
            y = df_init['Final_gen_state']==final_state
            w = df_init['Data_weight'].values
            if len(y) == 0:
                f = np.nan
            else:
                f = np.average(y, weights=w)
            probs_naive[(initial_state, final_state)] = f


    test_df['pAA'] = probs_naive[(0,0)]
    test_df['pAD'] = probs_naive[(0,1)]
    test_df['pAO'] = probs_naive[(0,2)]
    test_df['pDA'] = probs_naive[(1,0)]
    test_df['pDD'] = probs_naive[(1,1)]
    test_df['pOA'] = probs_naive[(2,0)]
    test_df['pOO'] = probs_naive[(2,2)]


    output_folder = Path(f"../Results/GAM/{folder_name}/Naive_model/")
    output_folder.mkdir(parents=True, exist_ok=True)
    output_file = output_folder / f"naive_failure_probabilities_{state}.csv"
    test_df.to_csv(output_file, index=False)
    

100%|██████████| 47/47 [00:12<00:00,  3.88it/s]
